Estudiantes: <br>
Tomás Rojas <br>
Matías Montagna <br>
Alonso Utreras

## Extracción de Datos:

Trabajaremos con los datos del [SINCA Temuco](https://sinca.mma.gob.cl/index.php/estacion/index/id/186). 

Este contiene información relevante como:
* MP 2,5 
* Presión atmosférica (Presión - hPa) 01-01-2005 5 mayo 2010
* Precipitaciones (Precipitaciones - mm)
* Temperatura ambiente (Temperatura - °C)
* Dirección del viento (Dir. viento - °)
* Velocidad del viento (Vel. viento - m/s)

Forma de los datos:
-------------------
Presión atmosférica
FECHA (YYMMDD);HORA (HHMM);;
050101;0100;0;
-------------------


Es importante que los datos estén bien correlaciones con sus fechas y horas respectivas.

In [ ]:
import csv
# from google.colab import drive
import pandas as pd 
from preprocessing import process_data


# drive.mount('/content/drive') # To obtain values from google drive.

dir_viento = pd.read_csv("/content/drive/My Drive/dir_viento.csv", delimiter=';') 
dir_viento.drop(columns='Unnamed: 3',inplace=True)

vel_viento = pd.read_csv("/content/drive/My Drive/vel_viento.csv", delimiter=';') 
vel_viento.drop(columns='Unnamed: 3',inplace=True)

vel_viento = pd.read_csv("/content/drive/My Drive/vel_viento.csv", delimiter=';') 
vel_viento.drop(columns='Unnamed: 3',inplace=True)

vel_viento = pd.read_csv("/content/drive/My Drive/vel_viento.csv", delimiter=';') 
vel_viento.drop(columns='Unnamed: 3',inplace=True)

vel_viento = pd.read_csv("/content/drive/My Drive/vel_viento.csv", delimiter=';') 
vel_viento.drop(columns='Unnamed: 3',inplace=True)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,FECHA (YYMMDD),HORA (HHMM),Unnamed: 2
0,40331,100,NaN
1,40331,200,NaN
2,40331,300,NaN
3,40331,400,NaN
4,40331,500,NaN


In [1]:
from preprocessing import process_data

dir_viento = process_data("./data/dir_viento.csv")
dir_viento

KeyError: "['Unnamed: 5'] not found in axis"

In [16]:
import pandas as pd 

vel_viento = pd.read_csv("vel_viento.csv", delimiter=';') 
vel_viento.drop(columns='Unnamed: 3', inplace=True)
vel_viento.rename(columns={'Unnamed: 2': 'wind_speed'}, inplace=True)

dir_viento = pd.read_csv("dir_viento.csv", delimiter=';') 
dir_viento.drop(columns='Unnamed: 3', inplace=True)
dir_viento.rename(columns={'Unnamed: 2': 'wind_dir'}, inplace=True)

p_atm = pd.read_csv("p_atm.csv", delimiter=';') 
p_atm.drop(columns='Unnamed: 3', inplace=True)
p_atm.rename(columns={'Unnamed: 2': 'atm_p'}, inplace=True)

precipitaciones = pd.read_csv("precipitaciones.csv", delimiter=';') 
precipitaciones.drop(columns='Unnamed: 3', inplace=True)
precipitaciones.rename(columns={'Unnamed: 2': 'precipitation'}, inplace=True)

temperatura_ambiente = pd.read_csv("temperatura_ambiente.csv", delimiter=';') 
temperatura_ambiente.drop(columns='Unnamed: 3', inplace=True)
temperatura_ambiente.rename(columns={'Unnamed: 2': 'ambient_t'}, inplace=True)

humedad_relativa = pd.read_csv("humedad_relativa.csv", delimiter=';') 
humedad_relativa.drop(columns='Unnamed: 3', inplace=True)
humedad_relativa.rename(columns={'Unnamed: 2': 'relative_humidity'}, inplace=True)


## Funciones para Trabajar los datos

In [17]:
# Data Transformation cells

def split_date(YYMMDD):
    """ Erase YYMMDD from DataFrame and add three columns for year, month and day. 
    """
    yymmdd = str(YYMMDD)
    if len(yymmdd) == 5:
        return [yymmdd[0], yymmdd[1:3], yymmdd[3::]]
    if len(yymmdd) == 6:
        return [yymmdd[0:2], yymmdd[2:4], yymmdd[4::]]

def split_date_dataframe(df):
    """Create a new dataframe with date"""
    dates = pd.DataFrame(df[df.columns[0]].apply(split_date).tolist(), columns=['Year', 'Month', 'Day'])
    df_whitout_ymd = df.drop(columns=[df.columns[0]])
    return dates.join(df_whitout_ymd)

def fix_hour(HHMM):
    """
    Two cases:
    1)  1 AM => 100
    2) 10 AM => 1000
    So we have to look at length first
    This is a branchless version of the following: 
    if len(hhmm) == 3:
        return hhmm[0:1]
    if len(hhmm) == 4:
        return hhmm[0:2]
    """
    hhmm = str(HHMM)
    return hhmm[0:len(hhmm)-2]

def fix_hour_dataframe(df):
    """Create a new dataframe but fix hour"""
    df['HORA (HHMM)'] = df['HORA (HHMM)'].apply(fix_hour)
    return df

def prepare_dataframe(df):
    """split date and fix hour """
    temp = split_date_dataframe(df)
    return fix_hour_dataframe(temp)

In [18]:
df = temperatura_ambiente
test = prepare_dataframe(df)
test

,Year,Month,Day,HORA (HHMM),ambient_t
0,14,03,24,1,NaN
1,14,03,24,2,NaN
2,14,03,24,3,NaN
3,14,03,24,4,NaN
4,14,03,24,5,NaN
...,...,...,...,...,...
55650,20,07,28,19,"5,55851"
55651,20,07,28,20,"3,69859"
55652,20,07,28,21,"2,70067"
55653,20,07,28,22,NaN


In [42]:
# df = dir_viento.merge(p_atm, on=['FECHA (YYMMDD)', 'HORA (HHMM)'], suffixes=('dir_viento', 'p_atm'))

### Generando el DataFrame final

In [14]:
dataframes = (dir_viento, p_atm, precipitaciones, temperatura_ambiente, humedad_relativa)
df = pd.DataFrame(dir_viento)
for _df in dataframes[1::]:
    print(_df)
    df = df.merge(_df, on=['FECHA (YYMMDD)', 'HORA (HHMM)'])
    print(df)

FECHA (YYMMDD)  HORA (HHMM) atm_p
0               50101          100     0
1               50101          200     0
2               50101          300     0
3               50101          400     0
4               50101          500     0
...               ...          ...   ...
46818          100505         1900   NaN
46819          100505         2000   NaN
46820          100505         2100   NaN
46821          100505         2200   NaN
46822          100505         2300   NaN

[46823 rows x 3 columns]
       FECHA (YYMMDD)  HORA (HHMM) wind_dir atm_p
0               50101          100    317,6     0
1               50101          200    244,4     0
2               50101          300    220,3     0
3               50101          400    220,1     0
4               50101          500    236,8     0
...               ...          ...      ...   ...
46818          100505         1900      NaN   NaN
46819          100505         2000      NaN   NaN
46820          100505         2100     

In [15]:
print(df)

Empty DataFrame
Columns: [wind_dir, atm_p, precipitation, ambient_t, FECHA (YYMMDD), HORA (HHMM), relative_humidity]
Index: []
